# Installing the necessary Python packages and importing essential libraries required for OCR, Face detection, text processing, and related tasks

In [ ]:
!pip install python-doctr -q
!pip install ultralytics  -q
!pip install Levenshtein -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os
import cv2
import numpy as np
import unicodedata
import string

from collections import OrderedDict
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageEnhance, ImageFilter

import re
import torch
from datetime import datetime
import json
import difflib

from doctr.models import ocr_predictor
from doctr.io import DocumentFile

import torch.nn as nn
from torchvision import transforms

# Defining Mean Functions for OCR and Text Classification

In [ ]:
def extract_coordinates(geometry):
    if hasattr(geometry, '__len__') and len(geometry) == 2:
        top_left, bottom_right = geometry
        x1, y1 = top_left
        x2, y2 = bottom_right
        coordinates = [
            [x1, y1],
            [x2, y1],
            [x2, y2],
            [x1, y2]
        ]
        return coordinates
    return []

def load_model(model_path, config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)

    predictor = ocr_predictor(
        det_arch=config['det_arch'],
        reco_arch=config['reco_arch'],
        pretrained=False
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor.load_state_dict(torch.load(model_path, map_location=device))
    predictor.eval()

    if torch.cuda.is_available():
        predictor = predictor.cuda()

    return predictor

def detect_text(predictor, image_path, text_threshold=0.7):
    doc = DocumentFile.from_images(image_path)

    with torch.no_grad():
        result = predictor(doc)

    boxes = []
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                coordinates = extract_coordinates(line.geometry)

                if len(coordinates) == 4:
                    box = np.array(coordinates)

                    if hasattr(line, 'confidence') and line.confidence >= text_threshold:
                        boxes.append(box)
                    elif not hasattr(line, 'confidence'):
                        boxes.append(box)

    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    pixel_boxes = []
    for box in boxes:
        pixel_box = box.copy()
        pixel_box[:, 0] *= width
        pixel_box[:, 1] *= height
        pixel_boxes.append(pixel_box)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return image, np.array(pixel_boxes), np.array(pixel_boxes)

def show_boxes(image, boxes):
    img_show = image.copy()
    for box in boxes:
        box = box.astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(img_show, [box], isClosed=True, color=(0, 255, 0), thickness=2)
    plt.figure(figsize=(12, 12))
    plt.imshow(img_show)
    plt.axis('off')
    plt.show()

In [ ]:
def crop_id_card(image, boxes):

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    crops = []

    for poly in boxes:
        poly = np.array(poly).astype(np.int32)

        # Create mask
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [poly], 255)

        # Apply mask
        masked = cv2.bitwise_and(image_rgb, image_rgb, mask=mask)

        # Get bounding box and crop
        x, y, w, h = cv2.boundingRect(poly)
        cropped = masked[y:y+h, x:x+w]
        crops.append(cropped)

    return crops


def plot_crops(crops, per_row=5, size=4):

    n = len(crops)
    rows = (n + per_row - 1) // per_row  # Ceiling division

    fig, axs = plt.subplots(rows, per_row, figsize=(per_row * size, rows * size))

    # Flatten axs for easy indexing, handle 1D or 2D array of axes
    axs = axs.flatten() if isinstance(axs, np.ndarray) else [axs]

    for i in range(len(axs)):
        if i < n:
            axs[i].imshow(crops[i])
            axs[i].set_title(f"Crop #{i+1}")
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Define the classifier saved model architecture
class LanguageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(LanguageClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 4 * 16, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
class_names = ['Arabic', 'Frensh']

def predict_language_cnn(image, model, class_names, transform):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    image_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

    predicted_class = class_names[predicted.item()]
    confidence = confidence.item() * 100

    return predicted_class, confidence

def classify_crops(crops):
    results = []

    for i, crop in enumerate(crops):
        try:
            prediction, confidence = predict_language_cnn(crop, model, class_names, test_transform)
            results.append((prediction, confidence))
            print(f"Crop {i+1}: {prediction} ({confidence:.2f}%)")

        except Exception as e:
            print(f"Error processing crop {i+1}: {str(e)}")
            results.append(("Error", 0))

    return results

In [ ]:
def plot_classification(original_image, crops, boxes, classification_results):
    img_with_boxes = original_image.copy()

    for i, (box, (prediction, confidence)) in enumerate(zip(boxes, classification_results)):
        box = box.astype(np.int32).reshape((-1, 1, 2))

        # Colors: Green for Arabic, Blue for English, Red for errors
        color = (0, 255, 0) if prediction == 'Arabic' else (255, 0, 0) if prediction == 'Frensh' else (0, 0, 255)

        # Draw thicker box
        cv2.polylines(img_with_boxes, [box], isClosed=True, color=color, thickness=3)

        # Get top-left corner for text placement
        x_min = np.min(box[:, 0, 0])
        y_min = np.min(box[:, 0, 1])

        # Create background for text
        label = f"{prediction} {confidence:.0f}%"
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]

        # Draw text background
        cv2.rectangle(img_with_boxes,
                     (x_min, y_min - text_size[1] - 5),
                     (x_min + text_size[0] + 10, y_min),
                     color, -1)

        # Draw text
        cv2.putText(img_with_boxes, label, (x_min + 5, y_min - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Language Classification Results')
    plt.show()

In [ ]:
def run_ocr(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with torch.no_grad():
        result = predictor([image_rgb])

    ocr_results = []
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                line_text = " ".join([word.value for word in line.words])

                if line.words:
                    x_coords = []
                    y_coords = []
                    for word in line.words:
                        for point in word.geometry:
                            x_coords.append(point[0])
                            y_coords.append(point[1])

                    x_min, x_max = min(x_coords), max(x_coords)
                    y_min, y_max = min(y_coords), max(y_coords)

                    ocr_results.append({
                        'text': line_text,
                        'bbox': (x_min, y_min, x_max, y_max),
                        'confidence': sum(word.confidence for word in line.words) / len(line.words)
                    })

    return ocr_results

def plot_ocr_results(image, ocr_results):
    img_with_text = image.copy()
    height, width = image.shape[:2]

    for i, result in enumerate(ocr_results):
        text = result['text']
        confidence = result['confidence']
        x_min, y_min, x_max, y_max = result['bbox']

        # Convert normalized coordinates to pixels
        x_min_px = int(x_min * width)
        y_min_px = int(y_min * height)
        x_max_px = int(x_max * width)
        y_max_px = int(y_max * height)

        # Draw bounding box
        cv2.rectangle(img_with_text, (x_min_px, y_min_px), (x_max_px, y_max_px), (0, 255, 255), 2)

        # Draw text background
        label = f"{text} ({confidence:.2f})"
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]

        cv2.rectangle(img_with_text,
                     (x_min_px, y_min_px - text_size[1] - 5),
                     (x_min_px + text_size[0] + 10, y_min_px),
                     (0, 255, 255), -1)

        # Draw text
        cv2.putText(img_with_text, label, (x_min_px + 5, y_min_px - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img_with_text, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('OCR Results - All Detected Text')
    plt.show()

def display_ocr_results(ocr_results):
    print("=== OCR RESULTS ===")
    print(f"Found {len(ocr_results)} text lines:")
    print("-" * 50)

    for i, result in enumerate(ocr_results, 1):
        print(f"{i}. '{result['text']}'")
        print(f"   Confidence: {result['confidence']:.3f}")
        print()

In [ ]:
def filter_ocr_with_classification(ocr_results, image, model, class_names, transform, confidence_threshold=0.7):
    filtered_texts = []
    height, width = image.shape[:2]

    for i, ocr_item in enumerate(ocr_results):
        text = ocr_item['text']
        ocr_confidence = ocr_item['confidence']
        x_min, y_min, x_max, y_max = ocr_item['bbox']

        # Skip empty text
        if not text.strip():
            continue

        # Convert normalized coordinates to pixels
        x_min_px = int(x_min * width)
        y_min_px = int(y_min * height)
        x_max_px = int(x_max * width)
        y_max_px = int(y_max * height)

        # Extract the region from image
        region = image[y_min_px:y_max_px, x_min_px:x_max_px]

        if region.size == 0:
            continue

        # Classify the region
        predicted_class, classification_confidence = predict_language_cnn(region, model, class_names, transform)

        # Keep text based on classification rules
        if ocr_confidence < confidence_threshold:
            # Low OCR confidence - use classification to decide
            if predicted_class == 'Arabic' and classification_confidence > 70:
                # Low confidence OCR + high confidence Arabic classification = discard
                print(f"Discarding: '{text}' (OCR: {ocr_confidence:.3f}, Class: {predicted_class} {classification_confidence:.1f}%)")
                continue
            else:
                # Keep if French or uncertain
                filtered_texts.append({
                    'text': text,
                    'ocr_confidence': ocr_confidence,
                    'classification': predicted_class,
                    'classification_confidence': classification_confidence,
                    'bbox': ocr_item['bbox'],
                    'region_id': i + 1
                })
        else:
            # High OCR confidence - keep regardless of classification
            filtered_texts.append({
                'text': text,
                'ocr_confidence': ocr_confidence,
                'classification': predicted_class,
                'classification_confidence': classification_confidence,
                'bbox': ocr_item['bbox'],
                'region_id': i + 1
            })

    return filtered_texts

def display_filtered_ocr_results(filtered_texts):
    print("=== FILTERED OCR RESULTS ===")
    print(f"Total texts: {len(filtered_texts)}")
    print("-" * 50)

    french_texts = [item for item in filtered_texts if item['classification'] == 'French']
    arabic_texts = [item for item in filtered_texts if item['classification'] == 'Arabic']
    other_texts = [item for item in filtered_texts if item['classification'] not in ['French', 'Arabic']]

    print(f"French texts: {len(french_texts)}")
    print(f"Arabic texts: {len(arabic_texts)}")
    print(f"Other/Uncertain: {len(other_texts)}")
    print("-" * 50)

    if french_texts:
        print("\n FRENCH TEXT (KEEP):")
        for result in french_texts:
            print(f"Region {result['region_id']}: '{result['text']}'")
            print(f"  OCR confidence: {result['ocr_confidence']:.3f}")
            print(f"  Classification: {result['classification']} ({result['classification_confidence']:.1f}%)")
            print()

    if arabic_texts:
        print("\n ARABIC TEXT (FILTERED):")
        for result in arabic_texts:
            print(f"Region {result['region_id']}: '{result['text']}'")
            print(f"  OCR confidence: {result['ocr_confidence']:.3f}")
            print(f"  Classification: {result['classification']} ({result['classification_confidence']:.1f}%)")
            print()

    if other_texts:
        print("\n OTHER TEXT (KEEP):")
        for result in other_texts:
            print(f"Region {result['region_id']}: '{result['text']}'")
            print(f"  OCR confidence: {result['ocr_confidence']:.3f}")
            print(f"  Classification: {result['classification']} ({result['classification_confidence']:.1f}%)")
            print()

def plot_final_results(image, filtered_texts):
    img_with_text = image.copy()
    height, width = image.shape[:2]

    for result in filtered_texts:
        text = result['text']
        x_min, y_min, x_max, y_max = result['bbox']
        language = result['classification']

        x_min_px = int(x_min * width)
        y_min_px = int(y_min * height)
        x_max_px = int(x_max * width)
        y_max_px = int(y_max * height)

        # Color based on language
        color = (0, 0, 255) if language == 'French' else (0, 255, 0)  # Red for French, Green for others

        cv2.rectangle(img_with_text, (x_min_px, y_min_px), (x_max_px, y_max_px), color, 2)

        label = f"{language}: {text}"
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]

        cv2.rectangle(img_with_text,
                     (x_min_px, y_min_px - text_size[1] - 5),
                     (x_min_px + text_size[0] + 10, y_min_px),
                     color, -1)

        cv2.putText(img_with_text, label, (x_min_px + 5, y_min_px - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img_with_text, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Final Filtered OCR Results')
    plt.show()


# Defining Mean Functions for Extracting Information from Front Side of ID Cards

In [ ]:
def find_name_blocks_idf(blocks):
    target_words = {"carte", "nationale", "identite"}
    header_block = None

    # Find header block containing any target words
    for block in blocks:
        text = block.get('text', '').lower()
        if any(word in text for word in target_words):
            header_block = block
            break
    if not header_block:
        return None, None

    # Calculate full horizontal range across all blocks
    all_x_mins = [b['bbox'][0] for b in blocks]
    all_x_maxs = [b['bbox'][2] for b in blocks]
    min_x = min(all_x_mins)
    max_x = max(all_x_maxs)
    horizontal_mid = (min_x + max_x) / 2

    hx_min, hy_min, hx_max, hy_max = header_block['bbox']

    # Filter candidate blocks:
    # - Vertically below the header (bbox[1] > header's bottom)
    # - Horizontally within the left half (bbox[0] < horizontal midpoint)
    candidates = [b for b in blocks
                  if b['bbox'][1] > hy_max and
                  b['bbox'][0] < horizontal_mid]

    # Sort candidates top-to-bottom by their y-min
    candidates.sort(key=lambda b: b['bbox'][1])

    if len(candidates) < 2:
        return None, None

    prenom = candidates[0]['text']
    nom = candidates[1]['text']

    return prenom, nom


def extract_cin_from_blocks_idf(blocks):
    pattern = re.compile(r'\b[A-Z]{1,3}\d{4,10}\b')
    results = []
    for block in blocks:
        text = block.get('text', '').upper()
        matches = pattern.findall(text)
        results.extend(matches)
    return results


def extract_dates_from_ocr_blocks_idf(blocks):
    date_pattern = re.compile(r'\b(\d{2})[\s./-](\d{2})[\s./-](\d{4})\b')
    year_pattern = re.compile(r'\b(19\d{2})\b')

    result = {
        "date_de_naissance": None,
        "valable_jusqua": None
    }

    texts = [block['text'] for block in blocks]
    found_dates = []

    for text in texts:
        matches = date_pattern.findall(text)
        raw_matches = date_pattern.finditer(text)
        for match in raw_matches:
            full_raw_date = match.group(0)  # e.g. '01.02.1980'
            found_dates.append(full_raw_date)
        if len(found_dates) >= 2:
            break

    if len(found_dates) >= 2:
        result["date_de_naissance"] = found_dates[0]
        result["valable_jusqua"] = found_dates[1]
        return result

    if len(found_dates) == 1:
        result["valable_jusqua"] = found_dates[0]

        for text in texts:
            clean_text = re.sub(r'[^\w\s]', '', text)
            year_matches = year_pattern.findall(clean_text)
            for y in year_matches:
                if y != found_dates[0][-4:]:
                    result["date_de_naissance"] = y
                    return result
        return result

    for text in texts:
        clean_text = re.sub(r'[^\w\s]', '', text)
        year_matches = year_pattern.findall(clean_text)
        if year_matches:
            result["date_de_naissance"] = year_matches[0]
            break

    return result



def normalize_text_idf(text):
    return re.sub(r'[^a-z]', '', text.lower())

def similarity_ratio_idf(a, b):
    a_norm = normalize_text_idf(a)
    b_norm = normalize_text_idf(b)
    if not a_norm or not b_norm:
        return 0
    matches = sum(1 for x, y in zip(a_norm, b_norm) if x == y)
    return matches / max(len(a_norm), len(b_norm))

def convert_date_to_iso_idf(date_str):
    date_pattern = re.compile(r'(\d{2})[\s./-](\d{2})[\s./-](\d{4})')
    m = date_pattern.search(date_str)
    if m:
        day, month, year = m.groups()
        try:
            return datetime.strptime(f'{day}-{month}-{year}', '%d-%m-%Y').date().isoformat()
        except:
            return None
    return None

def clean_ocr_blocks_idf(blocks):
    unwanted_texts = ['royaume du maroc', 'royaume maroc', 'carte nationale didentite',
                      "carte nationale d'identite", 'carte nationale identite', 'carte national didentite',
                      "carte national d'identite", 'carte national identite', 'specmen', 'specimen', 'ne le',
                      'née le', 'neé le', 'nee le', 'néle', 'néé le', 'nééle', 'neéle', 'du maroc', 'royaume du',
                      'nationale didentite', "nationale d'identite", 'nationale identite', 'national didentite',
                      "national d'identite", 'national identite', 'carte didentite', "carte d'identite", 'carte identite',
                      "maroc", "auime du maroc","N\"", 'Valable jusqu\'au', "royaume", "ROYAUUE DU MADOO"
                      ]



    dates = extract_dates_from_ocr_blocks_idf(blocks)
    dates_to_remove = {v for v in dates.values() if v is not None}

    cin_list = extract_cin_from_blocks_idf(blocks)
    cin_set = set(cin_list)

    cleaned_blocks = []
    for block in blocks:
        text = block.get('text', '').strip()
        if len(text) <= 1:
            continue

        if any(similarity_ratio_idf(text, unwanted) >= 0.9 for unwanted in unwanted_texts):
            continue

        # Check dates
        date_substrings = re.findall(r'\b\d{2}[\s./-]\d{2}[\s./-]\d{4}\b', text)
        block_dates = set()
        for ds in date_substrings:
            iso_date = convert_date_to_iso_idf(ds)
            if iso_date:
                block_dates.add(iso_date)

        if block_dates.intersection(dates_to_remove):
            continue

        # Check CIN pattern exact matches in this text (ignore case)
        text_upper = text.upper()
        if any(cin in text_upper for cin in cin_set):
            continue

        cleaned_blocks.append(block)

    return cleaned_blocks


def extract_names_with_validation_idf(blocks):
    cleaned_blocks = clean_ocr_blocks_idf(blocks)
    prenom, nom = find_name_blocks_idf(blocks)

    if prenom is None or nom is None:
        return None, None

    cleaned_texts = [block['text'].lower() for block in cleaned_blocks]

    def text_present(text):
        norm_text = normalize_text_idf(text)
        return any(norm_text in normalize_text_idf(ct) for ct in cleaned_texts)

    if not text_present(prenom) or not text_present(nom):
        return None, None

    return prenom, nom

def remove_name_blocks_idf(blocks):

    prenom, nom = extract_names_with_validation_idf(filtered_texts)
    prenom_norm = normalize_text_idf(prenom)
    nom_norm = normalize_text_idf(nom)

    filtered_blocks = []
    for block in blocks:
        text_norm = normalize_text_idf(block.get('text', ''))
        if text_norm == prenom_norm or text_norm == nom_norm:
            continue
        filtered_blocks.append(block)

    return filtered_blocks




def extract_lieu_idf(raw_blocks, clean_blocks):
    dates = extract_dates_from_ocr_blocks_idf(raw_blocks)
    ref_block = None

    # Try to find 'ne le' or similar in raw_blocks as reference block
    for block in raw_blocks:
        text_lower = block['text'].lower()
        if any(phrase in text_lower for phrase in ['neele', 'nele', 'ne le', 'né le', 'née le', 'ne lé', 'né lé', 'née lé', 'ne l', 'né l', 'née l', 'ne le.', 'né le.', 'née le.', 'ne-le', 'né-le', 'née-le', 'nè le', 'nè lé', 'nè-lé', 'nè-le', 'ne lè', 'né lè', 'née lè']):
            ref_block = block
            break

    # If not found, try to find block matching date_de_naissance from extracted dates
    if not ref_block and dates['date_de_naissance']:
        date_str = dates['date_de_naissance'].replace('-', '.')
        for block in raw_blocks:
            if date_str in block['text']:
                ref_block = block
                break

    if not ref_block:
        return None

    ref_x_min, ref_y_min, ref_x_max, ref_y_max = ref_block['bbox']

    candidates = [b for b in clean_blocks
                  if b['bbox'][0] <= ref_x_min + 0.05  # left aligned within 5%
                  and b['bbox'][1] > ref_y_max]        # below reference block

    if not candidates:
        return None

    candidates.sort(key=lambda b: b['bbox'][1])

    first_line = candidates[0]
    first_text = first_line['text']

    if len(candidates) > 1:
        second_line = candidates[1]

        vertical_gap = second_line['bbox'][1] - first_line['bbox'][3]
        horizontal_gap = abs(second_line['bbox'][0] - first_line['bbox'][0])

        if vertical_gap < 0.03 and horizontal_gap < 0.03 and len(first_text) > 5:
            return first_text + ' ' + second_line['text']

    return first_text




def extract_id_info(blocks):

    cin = extract_cin_from_blocks_idf(blocks)
    dates = extract_dates_from_ocr_blocks_idf(blocks)

    clean = clean_ocr_blocks_idf(blocks)
    prenom, nom = extract_names_with_validation_idf(blocks)
    clean = remove_name_blocks_idf(clean)

    lieu = extract_lieu_idf(blocks, clean)

    result = {
        "pays": "Maroc",
        "type_de_carte": "Carte Nationale d'Identité",
        "prenom": prenom,
        "Nom": nom,
        "date_de_naissance": dates.get("date_de_naissance"),
        "valable_jusqua": dates.get("valable_jusqua"),
        "lieu_de_naissance": lieu,
        "CIN": (cin[0] if len(cin) != 0 else None)
    }

    return result


# Defining Mean Functions for Extracting Information from Back Side of ID Cards

In [ ]:
def extract_4_info_idb(blocks):
    cin_pattern = re.compile(r'[A-Z]{1,3}\d{4,10}')
    code_pattern = re.compile(r'^[A-Z0-9]{7,10}$')
    num_etat_pattern = re.compile(r'\b\d+(?:/\d+)+\b')

    cin_candidates = []
    code_candidates = []

    for block in blocks:
        raw_text = block.get('text', '').upper()
        bbox = block.get('bbox', [])
        if not bbox or len(bbox) != 4:
            continue

        cin_matches = cin_pattern.findall(raw_text)
        for cin_match in cin_matches:
            if len(cin_match) < 9:
                cin_candidates.append((bbox, cin_match))

        text_stripped = raw_text.strip()
        if len(text_stripped) < 11 and code_pattern.fullmatch(text_stripped):
            code_candidates.append((bbox, text_stripped))

    if cin_candidates:
        cin_candidates.sort(key=lambda item: (item[0][1], item[0][0]))
        cin_result = [cin_candidates[0][1]]
        cin_bbox = cin_candidates[0][0]
    else:
        cin_result = []
        cin_bbox = None

    if code_candidates:
        code_candidates.sort(key=lambda item: (item[0][1], -item[0][0]))
        code_result = [code_candidates[-1][1]]
        code_bbox = code_candidates[-1][0]
    else:
        code_result = []
        code_bbox = None

    num_etat_civil_results = []
    if cin_bbox and code_bbox:
        left_x = cin_bbox[2]
        right_x = code_bbox[0]
        max_y = max(cin_bbox[1], code_bbox[1])

        candidates = []
        for block in blocks:
            bbox = block.get('bbox', [])
            if not bbox or len(bbox) != 4:
                continue
            x_min, y_min, _, _ = bbox
            if left_x <= x_min <= right_x and y_min <= max_y + 0.05:
                text = block.get('text', '').strip()
                matches = num_etat_pattern.findall(text)
                for match in matches:
                    candidates.append((bbox, match))

        if candidates:
            if len(candidates) == 1:
                num_etat_civil_results = [candidates[0][1]]
            else:
                center_x = (left_x + right_x) / 2
                center_y = max_y / 2

                def distance_to_center(bbox):
                    x_min, y_min, x_max, y_max = bbox
                    box_center_x = (x_min + x_max) / 2
                    box_center_y = (y_min + y_max) / 2
                    return ((box_center_x - center_x) ** 2 + (box_center_y - center_y) ** 2) ** 0.5

                candidates.sort(key=lambda c: distance_to_center(c[0]))
                num_etat_civil_results = [candidates[0][1]]

    for block in blocks:
        text = block.get('text', '').strip().upper()
        if re.fullmatch(r'SEXE\s*M', text):
            gender = 'M'
            break
        if re.fullmatch(r'SEXE\s*F', text):
            gender = 'F'
            break
    else:
        gender = None

    if gender is None and cin_bbox:
        cin_x_min, cin_y_min, cin_x_max, cin_y_max = cin_bbox
        for block in blocks:
            bbox = block.get('bbox', [])
            if not bbox or len(bbox) != 4:
                continue
            conf = block.get('conf', 0)
            if conf <= 0.8:
                continue
            x_min, y_min, x_max, y_max = bbox
            text = block.get('text', '').strip().upper()

            horizontally_aligned = (cin_x_min <= x_min <= cin_x_max) or (cin_x_min <= x_max <= cin_x_max)
            vertically_below = y_min > cin_y_max
            if horizontally_aligned and vertically_below:
                if text.startswith('FILE DE') or text.startswith('FILEDE'):
                    gender = 'F'
                    break
                if text.startswith('FILS DE') or text.startswith('FILSDE'):
                    gender = 'Male'
                    break

    if gender is None and code_bbox:
        code_x_min, code_y_min, code_x_max, code_y_max = code_bbox
        for block in blocks:
            bbox = block.get('bbox', [])
            if not bbox or len(bbox) != 4:
                continue
            x_min, y_min, x_max, y_max = bbox
            text = block.get('text', '').strip().upper()
            horizontally_aligned = (code_x_min <= x_min <= code_x_max) or (code_x_min <= x_max <= code_x_max)
            vertically_below = y_min > code_y_max
            if horizontally_aligned and vertically_below and len(text) <= 6:
                if re.search(r'\bSEXE\s*M\b', text):
                    gender = 'M'
                    break
                if re.search(r'\bSEXE\s*F\b', text):
                    gender = 'F'
                    break

        if gender is None:
            for block in blocks:
                bbox = block.get('bbox', [])
                if not bbox or len(bbox) != 4:
                    continue
                x_min, y_min, x_max, y_max = bbox
                text = block.get('text', '').strip().upper()
                horizontally_aligned = (code_x_min <= x_min <= code_x_max) or (code_x_min <= x_max <= code_x_max)
                vertically_below = y_min > code_y_max
                if horizontally_aligned and vertically_below and len(text) <= 6:
                    if re.search(r'\bM\b', text):
                        gender = 'Male'
                        break
                    if re.search(r'\bF\b', text):
                        gender = 'Female'
                        break

    return cin_result, code_result, num_etat_civil_results, gender

def extract_address_from_blocks_idb(blocks):
    address_keywords = {'ADRESSE', 'ADRESS', 'ADRES', 'ADESSE'}

    # First pass: Look for "adresse-like" word in the first two words of the block
    for block in blocks:
        text = block.get('text', '').upper().strip()
        words = re.split(r'\s+', text)
        first_two = words[:2]
        if any(word in address_keywords for word in first_two):
            return text

    # Fallback: Positional logic using CIN
    cin_pattern = re.compile(r'[A-Z]{1,3}\d{4,10}')
    cin_candidates = []

    for block in blocks:
        text = block.get('text', '').upper()
        bbox = block.get('bbox', [])
        if len(bbox) != 4:
            continue
        cin_matches = cin_pattern.findall(text)
        for match in cin_matches:
            cin_candidates.append((bbox, match))

    if not cin_candidates:
        return None

    # Get top-left-most CIN block
    cin_candidates.sort(key=lambda item: (item[0][1], item[0][0]))
    cin_bbox = cin_candidates[0][0]
    cin_x_min, _, _, cin_y_max = cin_bbox

    # Find blocks that are below CIN and roughly horizontally aligned
    aligned_blocks = []
    for block in blocks:
        bbox = block.get('bbox', [])
        if len(bbox) != 4:
            continue
        x_min, y_min, _, _ = bbox
        if abs(x_min - cin_x_min) < 0.1 and y_min > cin_y_max:
            aligned_blocks.append((y_min, block))

    # Sort by vertical position and take the third block (index 2)
    aligned_blocks.sort(key=lambda item: item[0])
    if len(aligned_blocks) >= 3:
        return aligned_blocks[2][1].get('text', '').strip()

    return None

def extract_nom_de_pere_mere_idb(blocks, cin_text):
    pere_keywords = ["FILS DE", "FILSDE", "FILEDE", "FILE DE"]
    mere_keywords = ["ET DE", "ETDE"]

    def starts_with_keyword(text, keywords):
        text_upper = text.upper().strip()
        return any(text_upper.startswith(k) for k in keywords)

    # Step 1: Find CIN block
    cin_text_upper = cin_text.upper()
    cin_block = next((b for b in blocks if cin_text_upper in b.get("text", "").upper()), None)

    cin_bbox = cin_block["bbox"]
    x_center = (cin_bbox[0] + cin_bbox[2]) / 2
    y_start = cin_bbox[3]

    # Step 2: Find blocks with high confidence and matching keywords
    pere_block = None
    mere_block = None
    for block in blocks:
        if block.get("ocr_confidence", 0) < 0.75:
            continue
        text = block.get("text", "").strip()
        if starts_with_keyword(text, pere_keywords) and not pere_block:
            pere_block = block
        elif starts_with_keyword(text, mere_keywords) and not mere_block:
            mere_block = block

    if pere_block and mere_block:
        return pere_block["text"], mere_block["text"]


    if not cin_block or "bbox" not in cin_block:
        return None, None
    # Step 3: Fallback using vertical line
    blocks_below = [
        b for b in blocks
        if b.get("bbox") and b["bbox"][1] > y_start
    ]
    blocks_below.sort(key=lambda b: b["bbox"][1])

    vertical_blocks = [
        b for b in blocks_below
        if b["bbox"][0] <= x_center <= b["bbox"][2]
    ]

    selected_blocks = vertical_blocks[:2]

    # Step 4: Use hybrid logic
    if pere_block or mere_block:
        if pere_block and not mere_block and len(selected_blocks) >= 2:
            return pere_block["text"], selected_blocks[1]["text"]
        elif mere_block and not pere_block and len(selected_blocks) >= 2:
            return selected_blocks[0]["text"], mere_block["text"]
        elif pere_block and not mere_block and len(selected_blocks) == 1:
            return pere_block["text"], None
        elif mere_block and not pere_block and len(selected_blocks) == 1:
            return None, mere_block["text"]

    # Step 5: If no keyword blocks at all, use both from vertical line
    if len(selected_blocks) >= 2:
        return selected_blocks[0]["text"], selected_blocks[1]["text"]
    else:
        return None, None


def extract_idback_info(blocks):

    cin_result, code_result, num_etat_civil_results, gender = extract_4_info_idb(blocks)
    extract_address_from_blocks_idb(blocks)
    try:
      nom_de_pere, nom_de_mere = extract_nom_de_pere_mere_idb(blocks, cin_result[0])
    except :
      nom_de_pere, nom_de_mere = None, None

    result = {
        "pays": "Maroc",
        "type_de_carte": "Carte Nationale d'Identité",
        "CIN": (cin_result[0] if cin_result else None),
        "Code": (code_result[0] if code_result else None),
        "Num_Civil": (num_etat_civil_results[0] if num_etat_civil_results else None),
        "Sexe": gender,
        "Pere": nom_de_pere,
        "Mere": nom_de_mere
    }

    return result


# Defining Mean Functions for Extracting Information from the Front Side of Driver’s License

In [ ]:
def extract_prenom_pc(blocks, filtered_texts):
    def normalize(text):
        text = unicodedata.normalize("NFD", text.lower().strip())
        return ''.join(c for c in text if unicodedata.category(c) != 'Mn')

    def starts_with_label(text, label):
        return normalize(text).startswith(normalize(label))

    def is_in_filtered(text):
        return any(text.strip() == ft['text'].strip() for ft in filtered_texts)

    def find_label(labels, position_check=None):
        for i, block in enumerate(blocks):
            for label in labels:
                if starts_with_label(block['text'], label):
                    if not position_check or position_check(block['bbox']):
                        return block
        return None

    def get_block_above(ref_block):
        ref_y = ref_block['bbox'][1]
        candidates = [
            block for block in blocks
            if block['bbox'][3] < ref_y and block['bbox'][0] < 0.5
        ]
        return sorted(candidates, key=lambda b: -b['bbox'][3])[0] if candidates else None

    def get_blocks_below(ref_block, count):
        ref_y = ref_block['bbox'][3]
        candidates = [
            block for block in blocks
            if block['bbox'][1] > ref_y and block['bbox'][0] < 0.5
        ]
        return sorted(candidates, key=lambda b: b['bbox'][1])[:count]

    # Step 1: Try "Nom"
    for label_group in [['Nom/', 'Nom', 'Nom...'], ['Prénom/', 'Prénom', 'Prénom...']]:
        label_type = 'Nom' if 'Nom' in label_group[0] else 'Prénom'
        label_block = find_label(label_group)
        if label_block:
            above = get_block_above(label_block)
            if above and is_in_filtered(above['text']):
                return above['text']

            # Check below if Prénom case
            if label_type == 'Prénom':
                below = get_blocks_below(label_block, count=2)
                if below:
                    best = max(below, key=lambda b: b['ocr_confidence'])
                    if is_in_filtered(best['text']):
                        return best['text']

    # Step 2: Fallback to CONDUIRE header
    def is_top_left(bbox):
        x, y = bbox[0], bbox[1]
        return x < 0.5 and y < 0.3

    conduire_block = find_label(['PERMIS DE CONDUIRE'], position_check=is_top_left)
    if conduire_block:
        below = get_blocks_below(conduire_block, count=4)
        for b in below:
            if b['ocr_confidence'] > 0.87 and is_in_filtered(b['text']):
                return b['text']

    return None

def extract_nom_from_blocks_pc(blocks):
    keywords_sets = [
        {"Date", "Naissance", "Lieu"},
        {"Date", "Naissance"},
        {"Naissance"}
    ]

    x_mins = [block['bbox'][0] for block in blocks]
    x_maxs = [block['bbox'][2] for block in blocks]
    card_center_x = (min(x_mins) + max(x_maxs)) / 2

    blocks_sorted = sorted(blocks, key=lambda b: b['bbox'][1])

    def contains_keywords(text, keywords):
        text_lower = text.lower()
        return all(k.lower() in text_lower for k in keywords)

    def is_date(text):
        date_regex = re.compile(r'\b\d{2}[\./-]\d{2}[\./-]\d{4}\b')
        return bool(date_regex.search(text))

    for keywords in keywords_sets:
        for i, block in enumerate(blocks_sorted):
            if contains_keywords(block['text'], keywords):
                keyword_ymin = block['bbox'][1]
                candidate_blocks = [
                    b for b in blocks_sorted
                    if ((b['bbox'][0] + b['bbox'][2]) / 2) < card_center_x and b['bbox'][1] < keyword_ymin
                ]
                if candidate_blocks:
                    candidate_blocks = sorted(candidate_blocks, key=lambda b: b['bbox'][1], reverse=True)
                    return candidate_blocks[0]['text']
                else:
                    return blocks_sorted[i-1]['text'] if i > 0 else None

    first_date_block = None
    for block in blocks_sorted:
        if is_date(block['text']):
            first_date_block = block
            break

    if first_date_block:
        date_ymin = first_date_block['bbox'][1]
        above_blocks = [
            b for b in blocks_sorted
            if ((b['bbox'][0] + b['bbox'][2]) / 2) < card_center_x and b['bbox'][1] < date_ymin
        ]
        if above_blocks:
            above_blocks = sorted(above_blocks, key=lambda b: b['bbox'][1], reverse=True)
            block_above_date = above_blocks[0]
            block_above_date_ymin = block_above_date['bbox'][1]
            above_above_blocks = [
                b for b in blocks_sorted
                if ((b['bbox'][0] + b['bbox'][2]) / 2) < card_center_x and b['bbox'][1] < block_above_date_ymin
            ]
            if above_above_blocks:
                above_above_blocks = sorted(above_above_blocks, key=lambda b: b['bbox'][1], reverse=True)
                return above_above_blocks[0]['text']

    return None

def normalize_text_pc(text):
    text = text.lower()
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join(text.split())
    return text

def extract_lieu_de_naissance_pc(boxes):
    variants = [
        "délivré à", "delivre a", "delivré a", "délivre a", "dellivre a",
        "delivreà", "delivrea", "delivréà", "délivréà",
        "delivré a:", "délivré à:", "délivré à .", "délivré à :",
    ]

    date_pattern = re.compile(r'\b\d{2}/\d{2}/\d{4}\b')
    date_boxes = [b for b in boxes if date_pattern.search(b['text'])]

    if len(date_boxes) < 2:
        return None

    date_boxes = sorted(date_boxes, key=lambda b: b['bbox'][1])
    first_date = date_boxes[0]
    second_date = date_boxes[1]

    y_first_bottom = first_date['bbox'][3]
    y_second_top = second_date['bbox'][1]
    x_second_left = second_date['bbox'][0]

    between_boxes = []
    for b in boxes:
        y_top, y_bottom = b['bbox'][1], b['bbox'][3]
        x_left = b['bbox'][0]
        text = b['text'].strip()
        conf = b['ocr_confidence']

        if y_bottom <= y_first_bottom or y_top >= y_second_top:
            continue

        if x_left >= x_second_left:
            continue

        if b == second_date:
            continue

        if y_bottom <= y_second_top and y_bottom > y_second_top - 0.05:
            text_norm = normalize_text_pc(text)
            if any(normalize_text_pc(v) in text_norm for v in variants):
                continue

        if len(text) <= 2:
            continue

        text_norm = normalize_text_pc(text)
        if any(normalize_text_pc(v) in text_norm for v in variants):
            continue

        between_boxes.append(b)

    high_conf_boxes = [b for b in between_boxes if b['ocr_confidence'] > 0.86]

    if len(high_conf_boxes) >= 2:
        high_conf_boxes = [b for b in high_conf_boxes if b.get('classification', '').lower() != 'arabic']

    if len(high_conf_boxes) == 0:
        candidate_boxes = between_boxes
    elif len(high_conf_boxes) == 1:
        candidate_boxes = high_conf_boxes
    else:
        candidate_boxes = high_conf_boxes

    if len(candidate_boxes) == 1:
        selected_text = candidate_boxes[0]['text']
    elif len(candidate_boxes) > 1:
        candidate_boxes = sorted(candidate_boxes, key=lambda b: b['ocr_confidence'], reverse=True)
        selected_text = candidate_boxes[0]['text']
    else:
        selected_text = None

    return selected_text

def remove_accents_pc(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

def extract_lieu_delivrance_pc(blocks):
    variants = [
        "délivré à", "delivre a", "delivré a", "délivre a", "dellivre a",
        "delivreà", "delivrea", "delivréà", "délivréà",
        "delivré a:", "délivré à:", "délivré à .", "délivré à :",
    ]

    # 1. First try exact match ignoring case (with accents)
    for block in blocks:
        original_text = block.get('text', '').strip()
        lower_text = original_text.lower()
        for variant in variants:
            if variant.lower() in lower_text:
                idx = lower_text.find(variant.lower())
                length = len(variant)
                after = original_text[idx + length:].strip(" :.")
                if after:
                    return after

    # 2. If no match found, try ignoring accents
    variants_no_accents = [remove_accents_pc(v).lower() for v in variants]

    for block in blocks:
        original_text = block.get('text', '').strip()
        text_no_accents = remove_accents_pc(original_text).lower()

        for variant_no_accents in variants_no_accents:
            if variant_no_accents in text_no_accents:
                idx = text_no_accents.find(variant_no_accents)
                length = len(variant_no_accents)
                after = original_text[idx + length:].strip(" :.")
                if after:
                    return after

    return None

def extract_license_number_pc(blocks):
    pattern = re.compile(r'\b\d{2}/\d{6}\b')

    for block in blocks:
        text = block.get('text', '')
        match = pattern.search(text)
        if match:
            return match.group(0)

    return None

def extract_cin_from_blocks_pc(blocks):
    pattern = re.compile(r'\b[A-Z]{1,3}\d{4,10}\b')
    results = []
    for block in blocks:
        text = block.get('text', '').upper()
        matches = pattern.findall(text)
        results.extend(matches)
    return results

def extract_dates_and_categories_pc(blocks):
    date_pattern = re.compile(r'\b(\d{2})[\s./-](\d{2})[\s./-](\d{4})\b')
    categories_list = {"AM", "A1", "A", "B", "C", "D", "EB", "EC", "ED"}

    result = {
        "date_de_naissance": None,
        "Delivre_le": None,
        "categories": []
    }

    # Filter blocks with decent OCR confidence
    filtered_blocks = [
        (block['text'], block['bbox'][1])
        for block in blocks
        if block.get('ocr_confidence', 0) > 0.7
    ]

    # Extract dates with y-position
    dates_with_positions = []
    for text, y in filtered_blocks:
        for match in date_pattern.finditer(text):
            dates_with_positions.append((match.group(0), y))
    dates_with_positions.sort(key=lambda x: x[1])  # sort top to bottom

    # Assign dates
    if len(dates_with_positions) >= 2:
        result["date_de_naissance"] = dates_with_positions[0][0]
        result["Delivre_le"] = dates_with_positions[1][0]
        delivre_le_y = dates_with_positions[1][1]
    elif len(dates_with_positions) == 1:
        result["Delivre_le"] = dates_with_positions[0][0]
        delivre_le_y = dates_with_positions[0][1]
    else:
        delivre_le_y = None

    # Extract categories from left-side blocks only (after Delivre_le)
    if delivre_le_y is not None:
        margin = 0.01
        left_below_blocks = [
            block for block in blocks
            if (
                block['bbox'][1] > delivre_le_y + margin and
                block.get('ocr_confidence', 0) > 0.6 and
                ((block['bbox'][0] + block['bbox'][2]) / 2) < 0.5  # left side
            )
        ]

        for block in left_below_blocks:
            words = re.findall(r'\b\w+\b', block['text'].upper())
            for w in words:
                if w in categories_list and w not in result["categories"]:
                    result["categories"].append(w)

    return result

def extract_permis_info(filtered_texts, ocr_results):
    result = {
        "pays": "Maroc",
        "type_de_carte": "PERMIS DE CONDUIRE",
        "prenom": None,
        "nom": None,
        "lieu_naissance": None,
        "lieu_delivrance": None,
        "numero_permis": None,
        "date_de_naissance": None,
        "delivre_le": None,
        "categories": [],
        "CIN": None
    }

    # Extract fields from OCR blocks
    result["prenom"] = extract_prenom_pc(ocr_results, filtered_texts)
    result["nom"] = extract_nom_from_blocks_pc(ocr_results)

    # Extract locations
    result["lieu_naissance"] = extract_lieu_de_naissance_pc(filtered_texts)
    result["lieu_delivrance"] = extract_lieu_delivrance_pc(filtered_texts)

    # Extract license number
    result["numero_permis"] = extract_license_number_pc(filtered_texts)

    # Extract dates and categories
    date_info = extract_dates_and_categories_pc(filtered_texts)
    result["date_de_naissance"] = date_info.get("date_de_naissance")
    result["delivre_le"] = date_info.get("Delivre_le")
    result["categories"] = date_info.get("categories", [])

    # Extract CIN
    CIN = extract_cin_from_blocks_pc(filtered_texts)
    result["CIN"] = CIN

    return result

# Defining Mean Functions for Extracting Information from PASSPORT

In [ ]:
def extract_passport_dates(blocks):
    date_pattern = re.compile(r'\b(\d{2})[/-](\d{2})[/-](\d{4})\b')

    date_blocks = []

    for block in blocks:
        if block.get("ocr_confidence", 0) < 0.4:
            continue

        text = block["text"]
        for match in date_pattern.finditer(text):
            date_str = match.group(0)
            day, month, year = match.groups()
            y_position = block["bbox"][1]
            date_blocks.append({
                "text": date_str,
                "year": int(year),
                "y": y_position
            })

    # Sort top-to-bottom by y-position
    date_blocks.sort(key=lambda x: x["y"])

    result = {
        "date_de_naissance": None,
        "date_de_delivrance": None,
        "date_dexpiration": None
    }

    # Try to find the first valid trio in chronological order
    for i in range(len(date_blocks) - 2):
        d1, d2, d3 = date_blocks[i], date_blocks[i+1], date_blocks[i+2]
        if d1["year"] < d2["year"] < d3["year"]:
            result["date_de_naissance"] = d1["text"]
            result["date_de_delivrance"] = d2["text"]
            result["date_dexpiration"] = d3["text"]
            return result

    return result


def extract_authorite(blocks, date_delivrance_text, y_tolerance=0.02):
    date_block = None
    for block in blocks:
        if block.get("ocr_confidence", 0) < 0.6:
            continue
        if date_delivrance_text in block["text"]:
            date_block = block
            break
    if not date_block:
        return None
    date_y = date_block["bbox"][1]
    date_x_max = date_block["bbox"][2]

    candidates = []
    for block in blocks:
        if block.get("ocr_confidence", 0) < 0.6:
            continue
        bbox = block["bbox"]
        block_y_top = bbox[1]
        block_y_bottom = bbox[3]
        if abs(block_y_top - date_y) < y_tolerance or abs(block_y_bottom - date_y) < y_tolerance:
            if bbox[0] > date_x_max:
                candidates.append(block)

    if not candidates:
        return None
    candidates.sort(key=lambda b: b["bbox"][0])
    return candidates[0]["text"]


def extract_passport_sexe(blocks, birth_date):
    if not birth_date:
        return None

    target_block = None
    margin_y = 0.01  # Vertical tolerance for alignment

    # Step 1: Find the block containing the birth date
    for block in blocks:
        if birth_date in block.get("text", ""):
            target_block = block
            break

    if not target_block:
        return None

    birth_y_min = target_block["bbox"][1]
    birth_y_max = target_block["bbox"][3]
    birth_x_min = target_block["bbox"][0]

    # Step 2: Scan all blocks to the left on the same line
    candidates = []
    for block in blocks:
        text = block.get("text", "").strip().upper()
        x_min, y_min, x_max, y_max = block["bbox"]

        # Must be aligned with the birth date row
        if y_min >= birth_y_min - margin_y and y_max <= birth_y_max + margin_y:
            # Must be to the left of birth date block
            if x_max <= birth_x_min:
                # Allow max 3 characters with M or F inside
                if len(text) <= 3 and any(letter in text for letter in ("M", "F")):
                    candidates.append((x_min, text))

    # Step 3: Return the closest valid gender marker to the birth date
    if candidates:
        candidates.sort(reverse=True)  # rightmost valid candidate first
        for _, text in candidates:
            if "M" in text:
                return "M"
            if "F" in text:
                return "F"

    return None

def extract_cin_and_passport_number(blocks):
    passport_regex = re.compile(r'\b[A-Z]{2}\d{7}\b')
    cin_regex = re.compile(r'\b[A-Z]{1,2}\d{4,7}\b')    #

    passport_candidates = []
    cin_candidates = []

    for block in blocks:
        text = block.get("text", "").strip().upper()
        if not text:
            continue

        x_min, y_min, x_max, y_max = block["bbox"]

        # Check for passport number (top right region)
        if passport_regex.fullmatch(text) and x_min >= 0.5 and y_min <= 0.4:
            passport_candidates.append((y_min, block))

        # Check for CIN (bottom left region)
        if cin_regex.fullmatch(text) and x_max <= 0.5 and y_max >= 0.6:
            cin_candidates.append((y_max, block))

    result = {
        "numero_passport": passport_candidates[0][1]["text"] if passport_candidates else None,
        "cin": cin_candidates[0][1]["text"] if cin_candidates else None
    }

    return result


def extract_passport_type(blocks, passport_number):
    passport_types = {"P", "PP", "PD", "PS", "S", "PL"}

    passport_number = passport_number.upper()
    target_block = None

    for block in blocks:
        text = block.get("text", "").strip().upper()
        if text == passport_number:
            target_block = block
            break

    if not target_block:
        return None

    y_center = (target_block["bbox"][1] + target_block["bbox"][3]) / 2
    x_min = target_block["bbox"][0]

    candidates = []

    for block in blocks:
        if block == target_block:
            continue
        y_block_center = (block["bbox"][1] + block["bbox"][3]) / 2
        x_max = block["bbox"][2]

        if abs(y_block_center - y_center) < 0.02:
            if x_max < x_min:
                text = block.get("text", "").strip().upper()
                if text in passport_types:
                    candidates.append((x_max, text))

    if candidates:
        candidates.sort(key=lambda x: x[0], reverse=True)
        return candidates[0][1]

    # Fallback: check last two lines of blocks for a starting string matching passport types
    blocks_sorted = sorted(blocks, key=lambda b: b["bbox"][1], reverse=True)
    last_two_lines = blocks_sorted[:2]

    for block in last_two_lines:
        text = block.get("text", "").strip().upper()
        if not text:
            continue
        prefix = text[:2]
        if prefix[0] == "P" and prefix not in passport_types:
            return "P"
        if prefix in passport_types:
            return prefix

    return None

import unicodedata

def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()

def extract_nom_passport(blocks, numero_passport=None):
    def find_blocks_containing(substring, accent_sensitive=True):
        results = []
        search_str = substring.lower() if accent_sensitive else normalize_text(substring)
        for block in blocks:
            text = block.get("text", "")
            target_text = text if accent_sensitive else normalize_text(text)
            if search_str in target_text:
                results.append(block)
        return results

    def left_side_blocks_below(target_block, count=2):
        y_max = target_block["bbox"][3]
        candidates = []
        for block in blocks:
            bx_min, by_min, bx_max, by_max = block["bbox"]
            if by_min > y_max and bx_max <= 0.5:
                candidates.append(block)
        candidates.sort(key=lambda b: b["bbox"][1])
        return candidates[:count]

    prenoms_blocks = [b for b in blocks if b.get("text","").lower().startswith("prénoms")]
    if prenoms_blocks:
        block = prenoms_blocks[0]
        above_candidates = [b for b in blocks if abs(b["bbox"][1] - block["bbox"][1]) < 0.05 and b["bbox"][3] < block["bbox"][1] and b["bbox"][0] < 0.5]
        if above_candidates:
            above_candidates.sort(key=lambda b: b["bbox"][1], reverse=True)
            return above_candidates[0]["text"]

    prenoms_blocks = find_blocks_containing("prenoms", accent_sensitive=False)
    if prenoms_blocks:
        block = prenoms_blocks[0]
        above_candidates = [b for b in blocks if abs(b["bbox"][1] - block["bbox"][1]) < 0.05 and b["bbox"][3] < block["bbox"][1] and b["bbox"][0] < 0.5]
        if above_candidates:
            above_candidates.sort(key=lambda b: b["bbox"][1], reverse=True)
            return above_candidates[0]["text"]

    given_blocks = []
    for b in blocks:
        text_norm = normalize_text(b.get("text",""))
        if "given" in text_norm:
            given_blocks.append(b)
    if given_blocks:
        block = given_blocks[0]
        above_candidates = [b for b in blocks if abs(b["bbox"][1] - block["bbox"][1]) < 0.05 and b["bbox"][3] < block["bbox"][1] and b["bbox"][0] < 0.5]
        if above_candidates:
            above_candidates.sort(key=lambda b: b["bbox"][1], reverse=True)
            return above_candidates[0]["text"]

    nom_blocks = []
    for b in blocks:
        text = b.get("text","").lower()
        if text.startswith("nom") or text.startswith("nom/") or text.startswith("noml") or text.startswith("nom/name"):
            nom_blocks.append(b)
    if nom_blocks:
        block = nom_blocks[0]
        below_candidates = left_side_blocks_below(block, count=2)
        if not below_candidates:
            return None
        if len(below_candidates) == 1:
            return below_candidates[0]["text"]
        c1, c2 = below_candidates[0], below_candidates[1]
        c1_conf = c1.get("ocr_confidence",0)
        c2_conf = c2.get("ocr_confidence",0)
        if c1_conf > 0.86 and c2_conf > 0.86:
            c1_class = c1.get("classification","").lower()
            c2_class = c2.get("classification","").lower()
            if c1_class == "french" and c2_class == "french":
                return c2["text"]
            elif c1_class == "french":
                return c1["text"]
            elif c2_class == "french":
                return c2["text"]
            else:
                return c1["text"]
        elif c1_conf > 0.86:
            return c1["text"]
        elif c2_conf > 0.86:
            return c2["text"]
        else:
            return c1["text"]

    if numero_passport:
        target_block = None
        for b in blocks:
            if b.get("text","").strip().upper() == numero_passport.upper():
                target_block = b
                break
        if target_block:
            y_max = target_block["bbox"][3]
            candidates = []
            for b in blocks:
                bx_min, by_min, bx_max, by_max = b["bbox"]
                if by_min > y_max and bx_max <= 0.5:
                    candidates.append(b)
            candidates.sort(key=lambda b: b["bbox"][1])
            candidates = candidates[:3]
            if not candidates:
                return None
            if len(candidates) == 1:
                return candidates[0]["text"]
            filtered = [c for c in candidates if c.get("ocr_confidence",0) > 0.86]
            if filtered:
                french_filtered = [c for c in filtered if c.get("classification","").lower() == "french"]
                if french_filtered:
                    return french_filtered[0]["text"]
                return filtered[0]["text"]
            return candidates[0]["text"]

    return None

def extract_prenom_passport(ocr_blocks, nom_value=None):

    # Helper function to find text with tolerance
    def find_text_pattern(pattern, blocks):
        results = []
        for block in blocks:
            text = block.get('text', '').upper()
            if re.search(pattern, text):
                results.append(block)
        return results

    # Improved spatial relationship function
    def get_boxes_in_relation(target_block, blocks, relation='above', x_tolerance=0.15, max_distance=0.3):
        target_bbox = target_block.get('bbox', (0, 0, 0, 0))
        target_center_x = (target_bbox[0] + target_bbox[2]) / 2

        candidates = []
        for block in blocks:
            if block == target_block:
                continue

            block_bbox = block.get('bbox', (0, 0, 0, 0))
            block_center_x = (block_bbox[0] + block_bbox[2]) / 2

            # Check if in same vertical column
            if abs(block_center_x - target_center_x) < x_tolerance:
                distance = None

                if relation == 'above' and block_bbox[3] < target_bbox[1]:
                    distance = target_bbox[1] - block_bbox[3]
                elif relation == 'below' and block_bbox[1] > target_bbox[3]:
                    distance = block_bbox[1] - target_bbox[3]

                if distance is not None and distance < max_distance:
                    candidates.append((block, distance))

        # Sort by distance and return
        return [block for block, dist in sorted(candidates, key=lambda x: x[1])]

    # TRY 1: Look for explicit "Prénoms" or "Given Names" label (MOST RELIABLE)
    prenoms_blocks = find_text_pattern(r'PRÉNOMS|PRENOMS|GIVEN.*NAMES', ocr_blocks)

    if prenoms_blocks:
        prenoms_block = prenoms_blocks[0]
        boxes_below = get_boxes_in_relation(prenoms_block, ocr_blocks, 'below')

        # Look for the actual given name below the label
        for box in boxes_below:
            text = box.get('text', '').strip()
            # More strict validation for given names
            if (2 <= len(text) <= 20 and
                re.match(r'^[A-Z][A-Za-z\.]*$', text) and
                not any(keyword in text.upper() for keyword in ['MAROC', 'MAROCAIN', 'PASSEPORT', 'NATIONAL', 'SEXE', 'DATE']) and
                not re.search(r'\d', text)):
                return text

    # TRY 2: Use nom value with BETTER filtering
    if nom_value:
        # Find the nom block
        nom_blocks = []
        clean_nom = re.sub(r'[^A-Z]', '', nom_value.upper())

        for block in ocr_blocks:
            clean_text = re.sub(r'[^A-Z]', '', block.get('text', '').upper())
            if clean_nom in clean_text and len(clean_text) >= len(clean_nom) * 0.8:
                nom_blocks.append(block)

        if nom_blocks:
            nom_block = nom_blocks[0]
            boxes_below = get_boxes_in_relation(nom_block, ocr_blocks, 'below')

            # Check if any box below looks like a given name with BETTER filtering
            for box in boxes_below:
                text = box.get('text', '').strip()

                # Skip nationality words and other non-name text
                if any(keyword in text.upper() for keyword in ['MAROCAIN', 'MAROCAINE', 'NATIONAL', 'SEXE', 'DATE']):
                    continue

                # Skip text with numbers or special characters (except dots for initials)
                if re.search(r'[^A-Za-z\.]', text) and not text.endswith('.'):
                    continue

                # Look for actual name text with reasonable length
                if (2 <= len(text) <= 20 and re.match(r'^[A-Z][A-Za-z\.]*$', text)):
                    return text

    # TRY 3: Find blocks containing "Nationalit" pattern
    nationalite_blocks = find_text_pattern(r'NATIONA(LIT|LITY|TÉ|TY)', ocr_blocks)
    if nationalite_blocks:
        nationalite_block = nationalite_blocks[0]
        boxes_above = get_boxes_in_relation(nationalite_block, ocr_blocks, 'above')

        # Look for boxes that are NOT the surname and look like given names
        for box in boxes_above:
            text = box.get('text', '').strip()

            # Skip if this is likely the surname (matches provided nom_value)
            if nom_value and nom_value.upper() in text.upper():
                continue

            # Skip label boxes and nationality words
            if any(keyword in text.upper() for keyword in ['PRÉNOMS', 'PRENOMS', 'GIVEN', 'NOM', 'NAME', 'MAROCAIN']):
                continue

            # Skip non-name text
            if re.match(r'^[^A-Za-z]*$', text) or re.search(r'\d', text):
                continue

            # This should be the given name!
            if 2 <= len(text) <= 20:
                return text

    # TRY 4: Direct search for name patterns (fallback)
    for block in ocr_blocks:
        text = block.get('text', '').strip()
        # Look for text that matches name patterns but excludes known non-name words
        if (2 <= len(text) <= 20 and
            re.match(r'^[A-Z][A-Za-z\.]*$', text) and
            not any(keyword in text.upper() for keyword in ['MAROC', 'PASSEPORT', 'NOM', 'PRÉNOMS', 'NATIONAL', 'SEXE', 'DATE', 'CARD']) and
            (not nom_value or nom_value.upper() not in text.upper())):
            return text

    return None

def get_vertical_strip_between_dates(ocr_data, date1, date2, overlap_margin=0.001):

    def find_bbox_by_text(target_text):
        for entry in ocr_data:
            if target_text.strip() in entry['text']:
                return entry['bbox']
        return None

    bbox1 = find_bbox_by_text(date1)
    bbox2 = find_bbox_by_text(date2)

    if not bbox1 or not bbox2:
        raise ValueError(f"One or both dates not found: '{date1}', '{date2}'")

    y1_top, y1_bottom = bbox1[1], bbox1[3]
    y2_top, y2_bottom = bbox2[1], bbox2[3]

    # Define the vertical range between dates
    y_top = min(y1_bottom, y2_bottom)
    y_bottom = max(y1_top, y2_top)

    def overlaps(a_top, a_bottom, b_top, b_bottom, margin=0.001):
        return not (a_bottom < b_top + margin or a_top > b_bottom - margin)

    # Filter entries that are within vertical range and NOT overlapping date lines
    filtered = []
    for entry in ocr_data:
        e_top, e_bottom = entry['bbox'][1], entry['bbox'][3]

        if y_top < e_top and e_bottom < y_bottom:
            # Fully between the dates
            filtered.append(entry)
        else:
            if overlaps(e_top, e_bottom, y1_top, y1_bottom, overlap_margin):
                continue
            if overlaps(e_top, e_bottom, y2_top, y2_bottom, overlap_margin):
                continue
            if y_top < e_top and e_bottom < y_bottom:
                filtered.append(entry)

    return filtered

def split_blocks_by_left_right(blocks):

    # Calculate x-centers for all blocks
    x_centers = []
    for block in blocks:
        x_min, _, x_max, _ = block['bbox']
        x_center = (x_min + x_max) / 2
        x_centers.append(x_center)

    # Compute the median x_center as the vertical split
    median_x = sorted(x_centers)[len(x_centers) // 2]

    left_blocks = []
    right_blocks = []

    for block, x_center in zip(blocks, x_centers):
        if x_center < median_x:
            left_blocks.append(block)
        else:
            right_blocks.append(block)

    return left_blocks, right_blocks

def extract_closest_field(texts):
    label_keywords = ['lieu', 'birth', 'domicile', 'date', 'naissanca', 'place', 'délivr']
    fallback_keyword = 'maroc'

    def is_close_word(word, keywords, max_dist=2):
        """Return True if word is within max_dist edit distance of any keyword"""
        return any(difflib.SequenceMatcher(None, word, kw).ratio() >= (1 - max_dist / max(len(word), len(kw))) for kw in keywords)

    # Only one box
    if len(texts) == 1:
        entry = texts[0]
        if entry.get('classification', '').lower() == 'french' or entry.get('ocr_confidence', 0) > 0.5:
            return [entry['text']]
        else:
            return None

    matched_labels = []

    # Try to find label using exact keyword match
    for entry in texts:
        text = entry['text'].lower()
        if any(kw in text for kw in label_keywords):
            matched_labels.append(entry)

    results = []

    # Use closest box to label(s)
    for label in matched_labels:
        lx_min, ly_min, lx_max, ly_max = label['bbox']
        label_center = ((lx_min + lx_max) / 2, (ly_min + ly_max) / 2)

        closest_candidate = None
        closest_distance = float('inf')

        for candidate in texts:
            if candidate == label:
                continue

            cx_min, cy_min, cx_max, cy_max = candidate['bbox']
            candidate_center = ((cx_min + cx_max) / 2, (cy_min + cy_max) / 2)

            distance = np.linalg.norm(np.array(candidate_center) - np.array(label_center))

            if distance < closest_distance:
                closest_distance = distance
                closest_candidate = candidate

        if closest_candidate:
            results.append(closest_candidate['text'])

    # Fallback – look for "maroc" in any text
    if not results:
        for entry in texts:
            if fallback_keyword in entry['text'].lower():
                results.append(entry['text'])
                break

    # Fuzzy label detection for 2-box edge case
    if not results and len(texts) == 2:
        for entry in texts:
            text_words = entry['text'].lower().split()
            for word in text_words:
                if is_close_word(word, label_keywords, max_dist=2):
                    # Return the *other* entry as the candidate
                    other = [e for e in texts if e != entry]
                    if other:
                        return [other[0]['text']]

    return results if results else None


def looks_like_address(text):
    """
    Return True if text contains symbols typically found in address blocks.
    """
    symbols = set('@[{!#%*]')
    return any(char in text for char in symbols)

def fuzzy_match_in_text(text, keywords, max_distance=2):
    """
    Check if any keyword approximately appears inside the text allowing max_distance errors.
    """
    text = text.lower()
    for keyword in keywords:
        keyword = keyword.lower()
        # Slide over the text with a window equal to keyword length and check fuzzy similarity
        length = len(keyword)
        for i in range(len(text) - length + 1):
            window = text[i:i+length]
            # Calculate Levenshtein-like distance using difflib SequenceMatcher ratio
            seq = difflib.SequenceMatcher(None, keyword, window)
            similarity = seq.ratio()
            # similarity of 1.0 means exact match; allow some fuzziness, e.g. >= 0.7
            if similarity >= 0.7:
                # Allow some differences in length by checking actual edit distance as well
                # But difflib ratio should be enough here
                return True
    return False

def extract_address(blocks, authorite_text):
    label_keywords = ['domicile', 'residence', 'authorit', 'authority', 'lieu', 'authori']

    # Locate and exclude the authorite block and anything below it
    authorite_box = None
    for block in blocks:
        if authorite_text and authorite_text.lower() in block['text'].lower():
            authorite_box = block
            break

    if authorite_box:
        _, a_ymin, _, _ = authorite_box['bbox']
        blocks = [b for b in blocks if b['bbox'][1] < a_ymin]

    if not blocks:
        return None

    # If only one block, return it if it looks like a valid address
    if len(blocks) == 1:
        b = blocks[0]
        if (
            b['classification'].lower() == 'french'
            or b['ocr_confidence'] > 0.5
            or looks_like_address(b['text'])
        ):
            return b['text']
        return None

    # Remove any label-like blocks using fuzzy matching on the full text
    filtered_blocks = []
    for b in blocks:
        if fuzzy_match_in_text(b['text'], label_keywords):
            # This block looks like a label, so skip it
            continue
        filtered_blocks.append(b)

    if not filtered_blocks:
        return None

    # Return cleaned result depending on how many blocks are left
    if len(filtered_blocks) == 1:
        return filtered_blocks[0]['text']
    elif len(filtered_blocks) == 2:
        return f"{filtered_blocks[0]['text']} {filtered_blocks[1]['text']}"
    else:
        # Sort left-to-right (x_min) if more than 2 blocks
        filtered_blocks.sort(key=lambda b: b['bbox'][0])
        return ' '.join(b['text'] for b in filtered_blocks)

def extract_passport_info(filtered_texts):
    # Extract all pieces
    cin_num = extract_cin_and_passport_number(filtered_texts) or {}
    dates = extract_passport_dates(filtered_texts) or {}
    authorite = extract_authorite(filtered_texts, dates.get('date_de_delivrance', '')) or ''
    sexe = extract_passport_sexe(filtered_texts, dates.get('date_de_naissance', '')) or ''
    passport_type = extract_passport_type(filtered_texts, cin_num.get('numero_passport', '')) or ''
    nom = extract_nom_passport(filtered_texts, cin_num.get('numero_passport', '')) or ''
    prenom = extract_prenom_passport(filtered_texts, nom_value=nom) or ''

    # Filter text vertically between dates, split into left/right blocks
    filt_text = get_vertical_strip_between_dates(filtered_texts, dates.get('date_de_naissance', ''), dates.get('date_de_delivrance', ''))
    left_blocks, right_blocks = split_blocks_by_left_right(filt_text)

    place_of_birth_list = extract_closest_field(left_blocks) or []
    place_of_birth = place_of_birth_list[0] if place_of_birth_list else ''

    address = extract_address(right_blocks, authorite) or ''

    # Compose flat dictionary
    result = {
        "pays": "Maroc",
        "type_de_carte": "PASSPORT",
        "Naionalite": "Marocaine",
        'numero_passport': cin_num.get('numero_passport', ''),
        'cin': cin_num.get('cin', ''),
        'date_de_naissance': dates.get('date_de_naissance', ''),
        'date_de_delivrance': dates.get('date_de_delivrance', ''),
        'date_dexpiration': dates.get('date_dexpiration', ''),
        'authorite': authorite,
        'sexe': sexe,
        'passport_type': passport_type,
        'nom': nom,
        'prenom': prenom,
        'place_of_birth': place_of_birth,
        'address': address,
    }

    return result


# DOC Type Classification

In [ ]:
def normalize(text):
    text = unicodedata.normalize("NFD", text.lower().strip())
    return ''.join(c for c in text if unicodedata.category(c) != 'Mn')

def classify_card_type(blocks):

    all_texts = [normalize(block.get('text', '')) for block in blocks]

    date_pattern = re.compile(r'\b\d{2}[\s./-]\d{2}[\s./-]\d{4}\b')
    date_count = sum(len(date_pattern.findall(text)) for text in all_texts)

    has_id_card = False
    has_permis = False
    has_passport = False

    for text in all_texts:
        if text == normalize("CARTE NATIONALE D'IDENTITE"):
            has_id_card = True
        elif "carte nationale" in text:
            has_id_card = True
        elif "valable jusqu" in text:
            has_id_card = True
        elif text == normalize("PERMIS DE CONDUIRE"):
            has_permis = True
        elif "conduire" in text:
            has_permis = True
        elif "passeport" in text:
            has_passport = True
        elif "kingdom of morocco" in text or "kingdom" in text:
            has_passport = True

    if has_passport:
        return "passport"
    if has_permis:
        return "permis_de_conduire"
    if date_count == 0:
        return "id_card_back"
    if date_count >= 3:
        return "passport"
    if has_id_card:
        return "id_card_front"
    if date_count <= 2:
        return "id_card_front"
    return "id_card_back"


# DOC Detection Model

In [ ]:
def detect_and_crop_id_cards(image_path, infer, cconfidence_threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    output_dict = infer(input_tensor)

    boxes = output_dict['detection_boxes'].numpy()[0]
    scores = output_dict['detection_scores'].numpy()[0]

    cropped_images = []
    height, width, _ = image_np.shape

    for box, score in zip(boxes, scores):
        if score < confidence_threshold:
            continue

        y1, x1, y2, x2 = box
        x1_px = int(x1 * width)
        y1_px = int(y1 * height)
        x2_px = int(x2 * width)
        y2_px = int(y2 * height)

        cropped_img = image.crop((x1_px, y1_px, x2_px, y2_px))
        cropped_images.append(cropped_img)

    return cropped_images

# Loading Models and Preparing Resources / Testing

In [ ]:
model_path_doctr = '/content/drive/MyDrive/poly-scan-ID/models/doctr_model.pth'
config_path_doctr = '/content/drive/MyDrive/poly-scan-ID/models/doctr_model_config.json'

predictor = load_model(model_path_doctr, config_path_doctr)

# Define the transforms used during training
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((64, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the trained model
model_path_classifier = '/content/drive/MyDrive/poly-scan-ID/models/language_classifier.pth'
num_classes = 2  # Ar and fr
model = LanguageClassifier(num_classes)
model.load_state_dict(torch.load(model_path_classifier, map_location=torch.device('cpu')))
model.eval()  # Set to evaluation mode


paths = [
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/id_card.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/id_card1.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/id_card2.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/id_card3.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/permi-1.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/id-back-1.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/id.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/p1.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/p2.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/p4.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/pss1.jpg",
    "/content/drive/MyDrive/CRAFT-ORC-ID/id-data/pss2.jpg"

]

import tensorflow as tf

# Load the model
#mdl = tf.saved_model.load('/content/id-card-detector/model/saved_model')
#nfer = mdl.signatures["serving_default"]

  0%|          | 0/59214222 [00:00<?, ?it/s]

In [ ]:
#image, boxes, polys = detect_text(predictor, paths[-2])
#show_boxes(image, boxes)

In [ ]:
#crops = crop_id_card(image, boxes)
#plot_crops(crops, per_row=8, size=1)

In [ ]:
#classification_results = classify_crops(crops)
#plot_classification(image, crops, boxes, classification_results)

In [ ]:
# Run OCR on full image
#ocr_results = run_ocr(image)

# Display text results
#display_ocr_results(ocr_results)

# Plot results on image
#plot_ocr_results(image, ocr_results)

In [ ]:
# Filter OCR results using classification
#filtered_texts = filter_ocr_with_classification(ocr_results, image, model, ['Arabic', 'French'], test_transform)

# Display results
#display_filtered_ocr_results(filtered_texts)

# Plot final results
#plot_final_results(image, filtered_texts)

# Get only French text for final output
#french_final_texts = [item for item in filtered_texts if item['classification'] == 'French']
#if french_final_texts:
    #print("\n=== FINAL FRENCH TEXT ===")
    #for result in french_final_texts:
        #print(f"'{result['text']}'")
#else:
    #print("\nNo French text found in final results")

# Face Detection Model (YOLO)

In [ ]:
from ultralytics import YOLO

def box_contains(boxA, boxB):
    return (boxA[0] <= boxB[0]) and (boxA[1] <= boxB[1]) and (boxA[2] >= boxB[2]) and (boxA[3] >= boxB[3])

def detect_face(image_path, model_path='/content/yolov8x.pt', conf=0.25,
                                        edge_threshold=20, max_width_ratio=0.55):
    model = YOLO(model_path)
    results = model(image_path, conf=conf)

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or unable to read: {image_path}")

    if len(results) == 0 or results[0].boxes is None or len(results[0].boxes) == 0:
        return None, None

    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    img_width = image.shape[1]
    img_center_x = img_width / 2

    to_remove = set()
    n = len(boxes)

    for i in range(n):
        x0, _, x1, _ = boxes[i]
        box_width = x1 - x0

        if x0 <= edge_threshold and x1 >= (img_width - edge_threshold):
            to_remove.add(i)
            continue

        if box_width > max_width_ratio * img_width:
            to_remove.add(i)
            continue

        for j in range(n):
            if i != j:
                if box_contains(boxes[i], boxes[j]):
                    to_remove.add(i)

    filtered_boxes = [boxes[i] for i in range(n) if i not in to_remove]

    if not filtered_boxes:
        return None, None

    best_face = None
    best_crop = None
    min_dist = float('inf')

    for box in filtered_boxes:
        x0, y0, x1, y1 = box
        face_center_x = (x0 + x1) / 2

        if face_center_x < img_center_x:
            dist = img_center_x - face_center_x
            if dist < min_dist:
                min_dist = dist
                best_face = [x0, y0, x1, y1]
                best_crop = image[y0:y1, x0:x1]

    return best_face, best_crop


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
#def plot_single_face(image_path, face, crop):
 #   image = cv2.imread(image_path)
 #   image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

 #   fig, ax = plt.subplots(1, figsize=(12, 10))
 #   ax.imshow(image_rgb)

 #   if face is not None:
 #       x0, y0, x1, y1 = face
 #       rect = patches.Rectangle((x0, y0), x1 - x0, y1 - y0,
 #                                linewidth=2, edgecolor='lime', facecolor='none')
 #       ax.add_patch(rect)

 #   plt.axis('off')
 #   plt.show()

 #   if crop is not None:
 #       crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
 #       plt.figure(figsize=(4, 4))
 #      plt.imshow(crop_rgb)
 #       plt.title("Selected Face")
 #      plt.axis('off')
 #       plt.show()

#path = paths[5]

#face_box, face_crop = detect_face(path)
#plot_single_face(path, face_box, face_crop)
